In [54]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as s
import numpy as np

# pd.set_option('display.max_rows', None) # code to not truncate dataframe when it is displayed



In [4]:
df = pd.read_csv("Resources/Data_Clean_hosp.csv")

df.head()

,Unnamed: 0,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,7184,AUT,Europe,Austria,2020-02-25,2.0,2.0,NaN,NaN,NaN,...,28.4,30.9,NaN,7.37,81.54,0.922,NaN,NaN,NaN,NaN
1,7185,AUT,Europe,Austria,2020-02-26,2.0,0.0,NaN,NaN,NaN,...,28.4,30.9,NaN,7.37,81.54,0.922,NaN,NaN,NaN,NaN
2,7186,AUT,Europe,Austria,2020-02-27,3.0,1.0,NaN,NaN,NaN,...,28.4,30.9,NaN,7.37,81.54,0.922,NaN,NaN,NaN,NaN
3,7187,AUT,Europe,Austria,2020-02-28,3.0,0.0,NaN,NaN,NaN,...,28.4,30.9,NaN,7.37,81.54,0.922,NaN,NaN,NaN,NaN
4,7188,AUT,Europe,Austria,2020-02-29,9.0,6.0,NaN,NaN,NaN,...,28.4,30.9,NaN,7.37,81.54,0.922,NaN,NaN,NaN,NaN


In [77]:
#changing date from string to datetime object
df["date"] = pd.to_datetime(df["date"],format='%Y-%m-%d')

type(df.iloc[0,4])

# df.iloc[0,4]

pandas._libs.tslibs.timestamps.Timestamp

In [78]:
#listing all columns

for _ in df.columns:
    print(_)

Unnamed: 0
iso_code
continent
location
date
total_cases
new_cases
new_cases_smoothed
total_deaths
new_deaths
new_deaths_smoothed
total_cases_per_million
new_cases_per_million
new_cases_smoothed_per_million
total_deaths_per_million
new_deaths_per_million
new_deaths_smoothed_per_million
reproduction_rate
icu_patients
icu_patients_per_million
hosp_patients
hosp_patients_per_million
weekly_icu_admissions
weekly_icu_admissions_per_million
weekly_hosp_admissions
weekly_hosp_admissions_per_million
new_tests
total_tests
total_tests_per_thousand
new_tests_per_thousand
new_tests_smoothed
new_tests_smoothed_per_thousand
positive_rate
tests_per_case
tests_units
total_vaccinations
people_vaccinated
people_fully_vaccinated
total_boosters
new_vaccinations
new_vaccinations_smoothed
total_vaccinations_per_hundred
people_vaccinated_per_hundred
people_fully_vaccinated_per_hundred
total_boosters_per_hundred
new_vaccinations_smoothed_per_million
stringency_index
population
population_density
median_age
age

In [79]:
df.columns

Index(['Unnamed: 0', 'iso_code', 'continent', 'location', 'date',
       'total_cases', 'new_cases', 'new_cases_smoothed', 'total_deaths',
       'new_deaths', 'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'new_tests', 'total_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smooth

In [80]:
df_vax_hosp_icu = df[['iso_code',
                    'continent', 
                    'location', 
                    'date',
                    'total_cases', 
                    'new_cases', 
                    'total_deaths',
                    'new_deaths',
                    'icu_patients',
                    'hosp_patients',
                    'weekly_icu_admissions',
                       'weekly_hosp_admissions',
                       'new_tests', 
                      'total_tests',
                       'positive_rate',
                      'total_vaccinations',
                       'people_vaccinated', 
                      'people_fully_vaccinated',
                      'total_boosters',
                       'new_vaccinations',
                       'population',
                      'hospital_beds_per_thousand',
                       'excess_mortality_cumulative_absolute',
                      'excess_mortality_cumulative',
                       'excess_mortality']]

In [81]:
df_vax_hosp_icu

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,icu_patients,hosp_patients,...,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,population,hospital_beds_per_thousand,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality
0,AUT,Europe,Austria,2020-02-25,2.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,9043072.0,7.37,NaN,NaN,NaN
1,AUT,Europe,Austria,2020-02-26,2.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,9043072.0,7.37,NaN,NaN,NaN
2,AUT,Europe,Austria,2020-02-27,3.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,9043072.0,7.37,NaN,NaN,NaN
3,AUT,Europe,Austria,2020-02-28,3.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,9043072.0,7.37,NaN,NaN,NaN
4,AUT,Europe,Austria,2020-02-29,9.0,6.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,9043072.0,7.37,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15413,USA,North America,United States,2021-10-16,44916737.0,32139.0,723765.0,397.0,16435.0,51706.0,...,407446961.0,218562924.0,188902483.0,10114824.0,876086.0,332915074.0,2.77,NaN,NaN,NaN
15414,USA,North America,United States,2021-10-17,44934357.0,17620.0,724110.0,345.0,16274.0,51361.0,...,408265959.0,218805579.0,189141481.0,10461286.0,818998.0,332915074.0,2.77,NaN,NaN,NaN
15415,USA,North America,United States,2021-10-18,45050910.0,116553.0,725835.0,1725.0,16375.0,51394.0,...,408797942.0,218973123.0,189292559.0,10681175.0,531983.0,332915074.0,2.77,NaN,NaN,NaN
15416,USA,North America,United States,2021-10-19,45132148.0,81238.0,728192.0,2357.0,16266.0,50832.0,...,409438987.0,219161368.0,189487793.0,10926564.0,641045.0,332915074.0,2.77,NaN,NaN,NaN


In [82]:
#listing all countries

countries = df["location"].unique()

print(f"This analysis involves data from {len(countries)} countries\n")

print(f"These countires are \n")

for x in countries:
    print(x)

This analysis involves data from 25 countries

These countires are 

Austria
Belgium
Bulgaria
Canada
Cyprus
Czechia
Denmark
Estonia
Finland
France
Germany
Ireland
Israel
Italy
Luxembourg
Malta
Netherlands
Portugal
Serbia
Slovenia
Spain
Sweden
Switzerland
United Kingdom
United States


In [102]:
df1 = df[["location","date","people_fully_vaccinated","new_cases","hosp_patients","icu_patients","new_deaths"]]

df1.head(10)

,location,date,people_fully_vaccinated,new_cases,hosp_patients,icu_patients,new_deaths
0,Austria,2020-02-25,NaN,2.0,NaN,NaN,NaN
1,Austria,2020-02-26,NaN,0.0,NaN,NaN,NaN
2,Austria,2020-02-27,NaN,1.0,NaN,NaN,NaN
3,Austria,2020-02-28,NaN,0.0,NaN,NaN,NaN
4,Austria,2020-02-29,NaN,6.0,NaN,NaN,NaN
5,Austria,2020-03-01,NaN,5.0,NaN,NaN,NaN
6,Austria,2020-03-02,NaN,4.0,NaN,NaN,NaN
7,Austria,2020-03-03,NaN,3.0,NaN,NaN,NaN
8,Austria,2020-03-04,NaN,8.0,NaN,NaN,NaN
9,Austria,2020-03-05,NaN,12.0,NaN,NaN,NaN


In [103]:
#groupby week and location

df1_week = df1.groupby(['location', pd.Grouper(key='date', freq='W-MON')]).mean().reset_index().sort_values(['location', 'date'])

# df1_week = df1.groupby(['location', pd.Grouper(key='date', freq='W-MON')])

df1_week 

,location,date,people_fully_vaccinated,new_cases,hosp_patients,icu_patients,new_deaths
0,Austria,2020-03-02,NaN,2.571429,NaN,NaN,NaN
1,Austria,2020-03-09,NaN,16.142857,NaN,NaN,NaN
2,Austria,2020-03-16,NaN,126.714286,NaN,NaN,0.600000
3,Austria,2020-03-23,NaN,493.714286,NaN,NaN,2.571429
4,Austria,2020-03-30,NaN,734.857143,NaN,NaN,12.428571
...,...,...,...,...,...,...,...
2243,United States,2021-09-27,1.829799e+08,117207.428571,77549.571429,22278.285714,2055.857143
2244,United States,2021-10-04,1.850356e+08,105054.000000,67578.714286,19533.428571,1801.428571
2245,United States,2021-10-11,1.866395e+08,86304.285714,59936.714286,17785.428571,1479.142857
2246,United States,2021-10-18,1.885608e+08,84932.142857,53596.000000,17006.428571,1662.571429


In [104]:
location = df1_week["location"]
date = df1_week["date"]
people_fully_vaccinated = df1_week["people_fully_vaccinated"]
new_cases = df1_week["new_cases"]
hosp_patients = df1_week["hosp_patients"]
icu_patients = df1_week["icu_patients"]
new_deaths = df1_week["new_deaths"]

In [42]:
# #TBC....

# # e.g.
# # subplots = 5
# # overall_title = "vaccinations vs hospitalisation"
# # x = loc/iloc
# # y = loc/iloc



# def plot_subplots(subplots, overall_title, x, y):
#     fig, axs = plt.subplots(subplots)
#     fig.suptitle(f"{overall_title}")
    
#     for _ in subplot:
#         axs[_].plot(x, y)

In [ ]:
fig, axs = plt.subplots(subplots)
fig.suptitle(f"{overall_title}")
    

